# Note
This Google Colab notebook is run in different sessions so the notebook is not suitable for running from start to finish in one go. This notebook serves more to "audit" my exploration.

In [ ]:
%%capture
!pip install huggingface-hub polars==1.9.0 -U

In [ ]:
%%capture
!huggingface-cli download HangenYuu/Steam_Games_Review --repo-type dataset --local-dir ./data

# Objectives
1. Normalize the data to make it look more like coming from an operational database.
2. Map the data to the correct schema to speed up initialization of the PosgreSQL database later.

# Transformation logic

1. Create Game ID in Games table.
2. Create tables for nested column (genres, developer, publisher).
3. Create bridge table for the nested column.
2. Create Users table with User ID & Username.
1. Use Game ID and User ID in Reviews table.



# Unnest `Games` table
**Note**: Exclude columns for system requirements as it's likely not be used in a SQL database (a document database such as MongoDB is suitable for the format). So they are dropped from the beginning.

In [3]:
import polars as pl
from pathlib import Path
import re

local_dir = Path("/content/data")

In [ ]:
def parse_reviews(value):
    if "%" in value:
        # Extract percentage and total number
        match = re.search(r"(\d+)% of ([\d,]+)", value)
        if match:
            percentage = int(match.group(1))
            total = int(match.group(2).replace(",", ""))
            return int((percentage / 100) * total)
    else:
        # Extract the number directly
        match = re.search(r"\(([\d,]+)\)", value)
        if match:
            return int(match.group(1).replace(",", ""))

df = pl.scan_csv(local_dir / "games_description.csv").drop("minimum_system_requirement", "recommend_system_requirement", "link", "short_description")
df = df.with_row_index("game_id", 1)
df = df.with_columns(
    pl.col("genres").str.replace_many(["]", "'", "["], "").str.split(", "),
    pl.col("long_description").str.strip_prefix("About This Game").str.strip_chars(),
    pl.col("number_of_english_reviews").str.replace_all(",", "").cast(pl.Int32),
    pl.col(["developer", "publisher"])
    .str.replace_many(["]", "'", "["], "")
    .str.split(", "),
    pl.when(pl.col("release_date").str.contains(r"\d{1,2} \w{3}, \d{4}"))
    .then(pl.col("release_date").str.to_date("%d %b, %Y", strict=False))
    .otherwise(pl.col("release_date").str.to_date("%b %Y", strict=False))
    .alias("release_date"),
    pl.col("number_of_reviews_from_purchased_people").map_elements(
        parse_reviews, return_dtype=pl.Int32
    ),
)

df = df.with_columns(
    pl.when(pl.col("number_of_reviews_from_purchased_people").is_null())
    .then(pl.lit("Not enough data"))
    .otherwise(pl.col("overall_player_rating"))
    .cast(pl.Categorical("lexical"))
    .alias("overall_player_rating"),
    pl.when(pl.col("number_of_reviews_from_purchased_people").is_null())
    .then(pl.col("overall_player_rating").str.extract(r"(\d+)").cast(pl.Int32))
    .otherwise(pl.col("number_of_reviews_from_purchased_people"))
    .alias("number_of_reviews_from_purchased_people"),
)

df.collect()

game_id,name,long_description,genres,release_date,developer,publisher,overall_player_rating,number_of_reviews_from_purchased_people,number_of_english_reviews
u32,str,str,list[str],date,list[str],list[str],cat,i32,i32
1,"""Black Myth: Wukong""","""Black Myth: Wukong is an actio…","[""Mythology"", ""Action RPG"", … ""Violent""]",2024-08-19,"[""Game Science""]","[""Game Science""]","""Overwhelmingly Positive""",654820,51931
2,"""Counter-Strike 2""","""For over two decades, Counter-…","[""FPS"", ""Shooter"", … ""Moddable""]",2012-08-21,"[""Valve""]","[""Valve""]","""Very Positive""",8313603,2258990
3,"""Warhammer 40,000: Space Marine…","""Embody the superhuman skill an…","[""Warhammer 40K"", ""Action"", … ""Futuristic""]",2024-09-09,"[""Saber Interactive""]","[""Focus Entertainment""]","""Very Positive""",50860,51920
4,"""Cyberpunk 2077""","""Cyberpunk 2077 is an open-worl…","[""Cyberpunk"", ""Open World"", … ""Immersive Sim""]",2020-12-10,"[""CD PROJEKT RED""]","[""CD PROJEKT RED""]","""Very Positive""",680264,324124
5,"""ELDEN RING""","""THE CRITICALLY ACCLAIMED FANTA…","[""Souls-like"", ""Dark Fantasy"", … ""Family Friendly""]",2022-02-25,"[""FromSoftware"", ""Inc.""]","[""FromSoftware"", ""Inc."", ""Bandai Namco Entertainment""]","""Very Positive""",705261,491741
…,…,…,…,…,…,…,…,…,…
286,"""MXGP PRO""","""The official video game of the…","[""Simulation"", ""Racing"", … ""Offroad""]",2018-06-29,"[""Milestone S.r.l.""]","[""Milestone S.r.l.""]","""Very Positive""",777,433
287,"""MXGP 2019 - The Official Motoc…","""The official Motocross Champio…","[""Simulation"", ""Racing"", … ""Multiplayer""]",2019-08-27,"[""Milestone S.r.l.""]","[""Milestone S.r.l.""]","""Very Positive""",772,446
288,"""Ride 2""","""Welcome to the temple of motor…","[""Racing"", ""Sports"", … ""Controller""]",2016-10-07,"[""Milestone S.r.l.""]","[""Milestone S.r.l.""]","""Very Positive""",2008,1045


In [ ]:
df.collect_schema().names()

['name',
 'long_description',
 'genres',
 'release_date',
 'developer',
 'publisher',
 'overall_player_rating',
 'number_of_reviews_from_purchased_people',
 'number_of_english_reviews']

In [ ]:
def create_dimension_n_mapping_tables(df: pl.LazyFrame, id_name: str, nested_column_name: str) -> tuple[pl.DataFrame, pl.DataFrame]:
    """
    Creates a game ID column in the original dataframe and generates genre dimension tables.

    Parameters:
    df: LazyFrame with a 'genres' column of type list[str]
    id_name: str, the name of the ID column in the new dimension table, without the "_id" suffix
    nested_column_name: str, the name of the nested column in the original dataframe

    Returns:
    tuple: (games_df, dimension_df, mapping_df)
        - dimension_df: DataFrame with genre_id and genre_name
        - mapping_df: DataFrame linking game_ids to genre_ids
    """
    exploded = (
        df.explode(nested_column_name)
    )

    dimension_df = (
        exploded
        .select(nested_column_name)
        .unique()
        .sort(nested_column_name)
        .with_row_index(f"{id_name}_id", 1)
        .rename({nested_column_name: id_name})
        .collect()
    )

    mapping_df = (
        exploded
        .join(
            dimension_df.lazy(),
            left_on=nested_column_name,
            right_on=id_name
        )
        .select(["game_id",f"{id_name}_id"])
        .collect()
    )

    return dimension_df, mapping_df

In [ ]:
genre_dimension_df, genre_mapping_df = create_dimension_n_mapping_tables(df, "genre", "genres")

In [ ]:
game_id = 1

original_genres = (
    df
    .filter(pl.col("game_id") == game_id)
    .select("genres")
    .collect()
    .item()
    .sort()
    .to_list()
)

mapped_genres = (
    genre_mapping_df
    .lazy()
    .filter(pl.col("game_id") == game_id)
    .join(
        genre_dimension_df.lazy(),
        left_on="genre_id",
        right_on="genre_id"
    )
    .select("genre")
    .collect()
    .get_column("genre")
    .sort()
    .to_list()
)

print("Original genres:", original_genres)
# e.g. ["FPS", "Moddable", "Shooter"]

print("Mapped genres:", mapped_genres)
# e.g. ["FPS", "Moddable", "Shooter"]

print("Matches:", set(original_genres) == set(mapped_genres))

Original genres: ['3D', 'Action', 'Action RPG', 'Action-Adventure', 'Adventure', 'Atmospheric', 'Combat', 'Dark Fantasy', 'Difficult', 'Drama', 'Fantasy', 'Hack and Slash', 'Music', 'Mythology', 'RPG', 'Singleplayer', 'Souls-like', 'Story Rich', 'Third Person', 'Violent']
Mapped genres: ['3D', 'Action', 'Action RPG', 'Action-Adventure', 'Adventure', 'Atmospheric', 'Combat', 'Dark Fantasy', 'Difficult', 'Drama', 'Fantasy', 'Hack and Slash', 'Music', 'Mythology', 'RPG', 'Singleplayer', 'Souls-like', 'Story Rich', 'Third Person', 'Violent']
Matches: True


In [ ]:
developer_dimension_df, developer_mapping_df = create_dimension_n_mapping_tables(df, "developer", "developer")
publisher_dimension_df, publisher_mapping_df = create_dimension_n_mapping_tables(df, "publisher", "publisher")

In [ ]:
df = df.drop("genres", "developer", "publisher")
df.collect()

game_id,name,long_description,release_date,overall_player_rating,number_of_reviews_from_purchased_people,number_of_english_reviews
u32,str,str,date,cat,i32,i32
1,"""Black Myth: Wukong""","""Black Myth: Wukong is an actio…",2024-08-19,"""Overwhelmingly Positive""",654820,51931
2,"""Counter-Strike 2""","""For over two decades, Counter-…",2012-08-21,"""Very Positive""",8313603,2258990
3,"""Warhammer 40,000: Space Marine…","""Embody the superhuman skill an…",2024-09-09,"""Very Positive""",50860,51920
4,"""Cyberpunk 2077""","""Cyberpunk 2077 is an open-worl…",2020-12-10,"""Very Positive""",680264,324124
5,"""ELDEN RING""","""THE CRITICALLY ACCLAIMED FANTA…",2022-02-25,"""Very Positive""",705261,491741
…,…,…,…,…,…,…
286,"""MXGP PRO""","""The official video game of the…",2018-06-29,"""Very Positive""",777,433
287,"""MXGP 2019 - The Official Motoc…","""The official Motocross Champio…",2019-08-27,"""Very Positive""",772,446
288,"""Ride 2""","""Welcome to the temple of motor…",2016-10-07,"""Very Positive""",2008,1045


## Final functions

In [4]:
def create_dimension_n_mapping_tables(df: pl.LazyFrame, id_name: str, nested_column_name: str) -> tuple[pl.LazyFrame, pl.LazyFrame]:
    """
    Creates a game ID column in the original dataframe and generates genre dimension tables.

    Parameters:
    df: LazyFrame with a 'genres' column of type list[str]
    id_name: str, the name of the ID column in the new dimension table, without the "_id" suffix
    nested_column_name: str, the name of the nested column in the original dataframe

    Returns:
    tuple: (games_df, dimension_df, mapping_df)
        - dimension_df: LazyFrame with id_name_id and id_name
        - mapping_df: LazyFrame linking game_ids to id_name_ids
    """
    exploded = (
        df.explode(nested_column_name)
    )

    dimension_df = (
        exploded
        .select(nested_column_name)
        .unique()
        .sort(nested_column_name)
        .with_row_index(f"{id_name}_id", 1)
        .rename({nested_column_name: id_name})
    )

    mapping_df = (
        exploded
        .join(
            dimension_df.lazy(),
            left_on=nested_column_name,
            right_on=id_name
        )
        .select(["game_id",f"{id_name}_id"])
    )

    return dimension_df, mapping_df

def process_games_description() -> None:
    def parse_reviews(value):
        if "%" in value:
            # Extract percentage and total number
            match = re.search(r"(\d+)% of ([\d,]+)", value)
            if match:
                percentage = int(match.group(1))
                total = int(match.group(2).replace(",", ""))
                return int((percentage / 100) * total)
        else:
            # Extract the number directly
            match = re.search(r"\(([\d,]+)\)", value)
            if match:
                return int(match.group(1).replace(",", ""))

    df = pl.scan_csv(local_dir / "games_description.csv").drop("minimum_system_requirement", "recommend_system_requirement", "link", "short_description")
    df = df.with_row_index("game_id", 1)
    df = df.with_columns(
        pl.col("genres").str.replace_many(["]", "'", "["], "").str.split(", "),
        pl.col("long_description").str.strip_prefix("About This Game").str.strip_chars(),
        pl.col("number_of_english_reviews").str.replace_all(",", "").cast(pl.Int32),
        pl.col(["developer", "publisher"])
        .str.replace_many(["]", "'", "["], "")
        .str.split(", "),
        pl.when(pl.col("release_date").str.contains(r"\d{1,2} \w{3}, \d{4}"))
        .then(pl.col("release_date").str.to_date("%d %b, %Y", strict=False))
        .otherwise(pl.col("release_date").str.to_date("%b %Y", strict=False))
        .alias("release_date"),
        pl.col("number_of_reviews_from_purchased_people").map_elements(
            parse_reviews, return_dtype=pl.Int32
        ),
    )

    df = df.with_columns(
        pl.when(pl.col("number_of_reviews_from_purchased_people").is_null())
        .then(pl.lit("Not enough data"))
        .otherwise(pl.col("overall_player_rating"))
        .cast(pl.Categorical("lexical"))
        .alias("overall_player_rating"),
        pl.when(pl.col("number_of_reviews_from_purchased_people").is_null())
        .then(pl.col("overall_player_rating").str.extract(r"(\d+)").cast(pl.Int32))
        .otherwise(pl.col("number_of_reviews_from_purchased_people"))
        .alias("number_of_reviews_from_purchased_people"),
    )
    genre_dimension_df, genre_mapping_df = create_dimension_n_mapping_tables(df, "genre", "genres")
    developer_dimension_df, developer_mapping_df = create_dimension_n_mapping_tables(df, "developer", "developer")
    publisher_dimension_df, publisher_mapping_df = create_dimension_n_mapping_tables(df, "publisher", "publisher")
    df = df.drop("genres", "developer", "publisher")
    destination_dir = local_dir / "processed"
    destination_dir.mkdir(exist_ok=True, parents=True)
    df.collect().write_parquet(destination_dir / "games_description.parquet")
    genre_dimension_df.collect().write_parquet(destination_dir / "genre_dimension.parquet")
    genre_mapping_df.collect().write_parquet(destination_dir / "genre_mapping.parquet")
    developer_dimension_df.collect().write_parquet(destination_dir / "developer_dimension.parquet")
    developer_mapping_df.collect().write_parquet(destination_dir / "developer_mapping.parquet")
    publisher_dimension_df.collect().write_parquet(destination_dir / "publisher_dimension.parquet")
    publisher_mapping_df.collect().write_parquet(destination_dir / "publisher_mapping.parquet")

process_games_description()

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
The token `Write Token` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate wh

In [6]:
!huggingface-cli upload HangenYuu/Steam_Games_Review /content/data/raw ./raw --repo-type=dataset

Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
Start hashing 3 files.
Finished hashing 3 files.
https://huggingface.co/datasets/HangenYuu/Steam_Games_Review/tree/main/./raw


In [7]:
!huggingface-cli upload HangenYuu/Steam_Games_Review /content/data/processed ./processed --repo-type=dataset

Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
Start hashing 7 files.
Finished hashing 7 files.
developer_dimension.parquet:   0% 0.00/3.88k [00:00<?, ?B/s]
developer_mapping.parquet:   0% 0.00/2.62k [00:00<?, ?B/s]

games_description.parquet:   0% 0.00/243k [00:00<?, ?B/s]



genre_dimension.parquet:   0% 0.00/4.28k [00:00<?, ?B/s]




Upload 7 LFS files:   0% 0/7 [00:00<?, ?it/s]


developer_mapping.parquet: 100% 2.62k/2.62k [00:00<00:00, 27.4kB/s]
genre_dimension.parquet: 100% 4.28k/4.28k [00:00<00:00, 40.2kB/s]
genre_mapping.parquet: 100% 8.81k/8.81k [00:00<00:00, 83.4kB/s]
developer_dimension.parquet: 100% 3.88k/3.88k [00:00<00:00, 31.0kB/s]
publisher_dimension.parquet:   0% 0.00/3.24k [00:00<?, ?B/s]
publisher_dimension.parquet: 100% 3.24k/3.24k [00:00<00:00, 47.3kB/s]
games_description.parquet: 100% 243k/243k [00:00<00:00, 800kB/s]





publisher_mapping.par

# Users Table from steam_game_reviews.csv
**Note**: The repo structure changed from the last operation, so the path changed.

In [1]:
%%capture
!pip install huggingface-hub[hf_transfer] polars==1.9.0 -U

In [2]:
%%capture
!huggingface-cli download HangenYuu/Steam_Games_Review --repo-type dataset --local-dir ./data

In [23]:
import polars as pl
from pathlib import Path
from datetime import datetime
import re

raw_dir = Path("/content/data/raw")
processed_dir = Path("/content/data/processed")

In [12]:
df = pl.scan_csv(raw_dir / "steam_game_reviews.csv", infer_schema_length=10000)

df = df.with_columns(
    pl.col("hours_played").str.replace_all(",", "").cast(pl.Float32),
    pl.col(["helpful", "funny"]).str.replace_all(",", "").cast(pl.Int64),
    pl.when(pl.col("recommendation") == "Recommended")
    .then(True)
    .otherwise(False)
    .alias("recommendation"),
    pl.when(
        ~(
            (pl.col("date").str.contains(r"\w+ \d{1,2},\s\d{4}"))
            | pl.col("date").str.contains(r"\d{1,2} \w+,\s\d{4}")
        )
    )
    .then(pl.concat_str([pl.col("date"), pl.lit(", " + str(datetime.now().year))]))
    .otherwise(pl.col("date"))
    .alias("date"),
    pl.when(pl.col("username").str.contains("\n"))
    .then(pl.col("username").str.extract(r"^(.*?)\n"))
    .otherwise(pl.col("username").fill_null("anonymous")),
)
df = df.with_columns(
    pl.when(pl.col("date").str.contains(r"\w+\s\d{1,2},\s\d{4}"))
    .then(pl.col("date").str.to_date("%B %d, %Y", strict=False))
    .when(pl.col("date").str.contains(r"\d{1,2}\s\w+,\s\d{4}"))
    .then(pl.col("date").str.to_date("%d %B, %Y", strict=False))
)

In [16]:
df.collect()

review,hours_played,helpful,funny,recommendation,date,game_name,username
str,f32,i64,i64,bool,date,str,str
"""The game itself is also super …",39.900002,1152,13,true,2024-09-14,"""Warhammer 40,000: Space Marine…","""Sentinowl"""
"""Never cared much about Warhamm…",91.5,712,116,true,2024-09-13,"""Warhammer 40,000: Space Marine…","""userpig"""
"""A salute to all the fallen bat…",43.299999,492,33,true,2024-09-14,"""Warhammer 40,000: Space Marine…","""Imparat0r"""
"""this game feels like it was ma…",16.799999,661,15,true,2024-09-14,"""Warhammer 40,000: Space Marine…","""Fattest_falcon"""
"""Reminds me of something I've l…",24.0,557,4,true,2024-09-12,"""Warhammer 40,000: Space Marine…","""Jek"""
…,…,…,…,…,…,…,…
"""2022 Early Access Review Loads…",4.2,1,0,true,2022-08-04,"""Turbo Golf Racing""","""Fatal Exit"""
"""2022 Early Access Review Great…",8.5,1,0,true,2022-08-04,"""Turbo Golf Racing""","""cleybaR"""
"""2022 Early Access Review Excel…",83.300003,2,0,true,2022-08-04,"""Turbo Golf Racing""","""Sim"""


In [18]:
games_description_df = pl.scan_parquet(processed_dir / "games_description.parquet").select("game_id", "name")
games_description_df.collect()

game_id,name
u32,str
1,"""Black Myth: Wukong"""
2,"""Counter-Strike 2"""
3,"""Warhammer 40,000: Space Marine…"
4,"""Cyberpunk 2077"""
5,"""ELDEN RING"""
…,…
286,"""MXGP PRO"""
287,"""MXGP 2019 - The Official Motoc…"
288,"""Ride 2"""


In [20]:
users_df = (
    df
    .select("username")
    .unique()
    .with_row_index(f"user_id", 1)
)
users_df.collect()

user_id,username
u32,str
1,"""Phoenix Oath"""
2,"""junesky"""
3,"""inflammab1e"""
4,"""beserker"""
5,"""The Denogginizer"""
…,…
701969,"""Walrus Clan (Wally)"""
701970,"""baldguy069"""
701971,"""alwaysbeyou91"""


In [22]:
updated_reviews_df = (
    df
    # Join with users to get user_id
    .join(
        users_df,
        left_on="username",
        right_on="username"
    )
    # Join with games to get game_id
    .join(
        games_description_df,
        left_on="game_name",
        right_on="name"
    )
    # Drop columns replaced with IDs
    .select(pl.all().exclude("game_name", "username"))
)
updated_reviews_df.collect()

review,hours_played,helpful,funny,recommendation,date,user_id,game_id
str,f32,i64,i64,bool,date,u32,u32
"""The game itself is also super …",39.900002,1152,13,true,2024-09-14,147708,3
"""Never cared much about Warhamm…",91.5,712,116,true,2024-09-13,562937,3
"""A salute to all the fallen bat…",43.299999,492,33,true,2024-09-14,408104,3
"""this game feels like it was ma…",16.799999,661,15,true,2024-09-14,436735,3
"""Reminds me of something I've l…",24.0,557,4,true,2024-09-12,603056,3
…,…,…,…,…,…,…,…
"""2021 Early Access Review nooii…",124.0,0,0,true,2021-12-20,57422,283
"""2021 Early Access Review great…",750.299988,0,0,true,2021-12-19,327496,283
"""2021 Early Access Review Great…",212.399994,0,0,true,2021-12-18,650298,283


## Final functions

In [24]:
import polars as pl
from pathlib import Path
from datetime import datetime
import re


def process_games_reviews() -> None:
    raw_dir = Path("/content/data/raw")
    processed_dir = Path("/content/data/processed")

    df = pl.scan_csv(raw_dir / "steam_game_reviews.csv", infer_schema_length=10000)
    df = df.with_columns(
        pl.col("hours_played").str.replace_all(",", "").cast(pl.Float32),
        pl.col(["helpful", "funny"]).str.replace_all(",", "").cast(pl.Int64),
        pl.when(pl.col("recommendation") == "Recommended")
        .then(True)
        .otherwise(False)
        .alias("recommendation"),
        pl.when(
            ~(
                (pl.col("date").str.contains(r"\w+ \d{1,2},\s\d{4}"))
                | pl.col("date").str.contains(r"\d{1,2} \w+,\s\d{4}")
            )
        )
        .then(pl.concat_str([pl.col("date"), pl.lit(", " + str(datetime.now().year))]))
        .otherwise(pl.col("date"))
        .alias("date"),
        pl.when(pl.col("username").str.contains("\n"))
        .then(pl.col("username").str.extract(r"^(.*?)\n"))
        .otherwise(pl.col("username").fill_null("anonymous")),
    )
    df = df.with_columns(
        pl.when(pl.col("date").str.contains(r"\w+\s\d{1,2},\s\d{4}"))
        .then(pl.col("date").str.to_date("%B %d, %Y", strict=False))
        .when(pl.col("date").str.contains(r"\d{1,2}\s\w+,\s\d{4}"))
        .then(pl.col("date").str.to_date("%d %B, %Y", strict=False))
    )

    games_description_df = pl.scan_parquet(
        processed_dir / "games_description.parquet"
    ).select("game_id", "name")

    users_df = df.select("username").unique().with_row_index(f"user_id", 1)

    updated_reviews_df = (
        df
        # Join with users to get user_id
        .join(users_df, left_on="username", right_on="username")
        # Join with games to get game_id
        .join(games_description_df, left_on="game_name", right_on="name")
        # Drop columns replaced with IDs
        .select(pl.all().exclude("game_name", "username"))
    )
    updated_reviews_df.collect().write_parquet(processed_dir / "games_reviews.parquet")
    users_df.collect().write_parquet(processed_dir / "users_mapping.parquet")

process_games_reviews()

In [26]:
!export HF_HUB_ENABLE_HF_TRANSFER=1
!huggingface-cli login
!huggingface-cli upload HangenYuu/Steam_Games_Review /content/data/raw ./raw --repo-type=dataset
!huggingface-cli upload HangenYuu/Steam_Games_Review /content/data/processed ./processed --repo-type=dataset


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write

# Ranking table
This table is very simple.

In [36]:
import polars as pl
from pathlib import Path


def process_games_ranking() -> None:
    raw_dir = Path("/content/data/raw")
    processed_dir = Path("/content/data/processed")

    genre_dimension_df = pl.scan_parquet(processed_dir / "genre_dimension.parquet")
    games_description_df = pl.scan_parquet(processed_dir / "games_description.parquet").select("game_id", "name")

    schema = pl.Schema(
        {
            "game_name": pl.String(),
            "genre": pl.String(),
            "rank_type": pl.String(),
            "rank": pl.UInt8(),
        }
    )

    df = pl.scan_csv(raw_dir / "games_ranking.csv", schema=schema)
    rank_df = df.select("rank_type").unique().with_row_index("rank_type_id", 1)

    df = (
        df.join(
            games_description_df,
            left_on="game_name",
            right_on="name",
        )
        .join(genre_dimension_df, left_on="genre", right_on="genre")
        .join(rank_df, left_on="rank_type", right_on="rank_type")
    )

    df.select(
        pl.all().exclude("game_name", "genre", "rank_type")
    ).collect().write_parquet(processed_dir / "games_ranking.parquet")
    rank_df.collect().write_parquet(processed_dir / "rank_dimension.parquet")

process_games_ranking()

In [37]:
pl.read_parquet(processed_dir / "games_ranking.parquet")

rank,game_id,genre_id,rank_type_id
u8,u32,u32,u32
1,2,16,2
2,3,16,2
3,4,16,2
4,1,16,2
5,5,16,2
…,…,…,…
36,223,295,1
37,113,295,1
38,189,295,1


In [38]:
!export HF_HUB_ENABLE_HF_TRANSFER=1
!huggingface-cli login
!huggingface-cli upload HangenYuu/Steam_Games_Review /content/data/raw ./raw --repo-type=dataset
!huggingface-cli upload HangenYuu/Steam_Games_Review /content/data/processed ./processed --repo-type=dataset


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write